## Hiring problem

A company that deals with restorations must plan recruitment actions for the next five months.

At first, the company has 20 skilled workers; every skilled worker provides 150 hours of work a
month and receives a monthly salary of EUR 2,000.

A newly recruited worker during the first month of service receives a salary of EUR 1,000 but he
does not provide any useful work in practice; for this first month he is assisted by an experienced
worker to teach the job.

Every skilled worker who performs coaching provides for 70 hours of work per month instead of
150 (due to time spent on coaching).

After one month of training newly hired workers become experts, with equal ability to work and
equal salary. 

The amount of hours of work to be covered for the next five months are 2000, 4000,
5000, 7000, 3500, respectively. 

The management has also imposed restrictions on recruitment: in
the next five months the company can hire a maximum of 20, 10, 15, 10 and 5 workers
respectively.

Use mip to formulate and solve the problem of finding a suitable recruitment
plan to cover the hours of work for the given period at minimum staff cost.

### Parameters:
 * $M$ is the number of months to schedule
 * $r_i$ is the number of hours to cover in month $i \in \{1, . . . , M\}$
 * $q_i$ is the maximum number of workers that can be recruited in month i
 * $N$ is the number of skilled workers at the beginning of the scheduling horizon
 * $h$ is the number of hours provided by a skilled worker
 * $k$ is the number of hours provided by a skilled worker that assists a newly recruited worker
 * $c$ is the salary of a skilled worker
 * $g$ is the salary of a newly recruited worker

### Variables:
 * $x_i \in \mathbb Z_+$ is the number of workers hired in month i
 * $y_i \in \mathbb Z_+$ is the number of skilled workers available in month i

### Model:
$$
\begin{array}{llll}
\min & \sum_{i=1}^{M}{(c_i y_i + g x_i)} \\
\textrm{s.t.} & h(y_i - x_i) + kx_i \ge r_i &\qquad \forall i \in \{1,...,M\} \\
              & x_i \le q_i &\qquad \forall i \in \{1,...,M\}\\
              & x_i \le y_i &\qquad \forall i \in \{1,...,M\}\\
              & y_i = y_{i-1} + x_{i-1} &\qquad \forall i \in \{2,...,M\}\\
              & y_1 = N &\qquad \\
              & x_i, y_i \in \mathbb Z_+ &\qquad \forall i \in \{2,...,M\}
\end{array}
$$

In [1]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 427 kB 4.1 MB/s 
  Attempting uninstall: cffi
    Found existing installation: cffi 1.15.1
    Uninstalling cffi-1.15.1:
      Successfully uninstalled cffi-1.15.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 3.5 MB/s 


In [2]:
M = 5
r = [2000, 4000, 5000, 7000, 3500]
q = [20, 10, 15, 10, 5]

N = 20
h = 150
k = 70

c = 2000
g = 1000

In [4]:
import mip

m = mip.Model()

x = [m.add_var(var_type=mip.INTEGER) for i in range(M)]
y = [m.add_var(var_type=mip.INTEGER) for i in range(M)]

for i in range(M):
    m.add_constr(h * (y[i] - x[i]) + k * x[i] >= r[i])
    m.add_constr(x[i] <= q[i])
    m.add_constr(x[i] <= y[i])

    if i == 0:
        m.add_constr(y[i] == N)
    else:
        m.add_constr(y[i] == y[i-1] + x[i-1])

m.objective = mip.minimize(c * mip.xsum(y) + g * mip.xsum(x))

m.optimize()
print(m.objective_value)
print([x[i].x for i in range(M)])
print([y[i].x for i in range(M)])

395000.0
[11.0, 8.0, 8.0, 0.0, 0.0]
[20.0, 31.0, 39.0, 47.0, 47.0]
